# <div align="center"> Lenguaje de programacion Python </div>
<h4 align="center">Actividad 7 - Archivos </h4>
<h4 align="center">Cornejo Pacheco Francisco - 375037</h4><h4 align="center">Profesor : Pedro Nuñez Yepiz</h4>
<h4 align="center">31 - OCT - 2024</h4>
<h4>Repositorio: https://github.com/FrankSkep/lenguaje-python</h4>

## Ejercicio

REALIZA UN PROGRAMA QUE UTILICE UNA LISTA DE DICCIONARIOS

MENU
1. Agregar (automático 10)
2. Eliminar {ID}
3. Imprimir lista ( tabla)
4. Buscar {ID}
5. Buscar {appat} todas las coincidencias
6. Ordenar {ID}
7. Generar archivo (preguntar nombre del archivo)
  - a) excel
  - b) txt
  - c) cvs
  - d) MarkDown
8. Cargar archivo
9. Imprimir archivo
10. Borrar Toda la lista
0. SALIR

- Preguntar siempre si esta seguro eliminar, No ordenar si ya esta Ordenado, decir si lista vacía, no existe o mostrar si está la búsqueda, No se permiten ID repetidos.

## Configuracion para trabajar con archivos en Google Colab

In [64]:
# Montar drive
from google.colab import drive
import os

ruta = '/content/drive'
drive.mount(ruta)

# Folder de trabajo para la actividad
ruta_folder = ruta + '/MyDrive/L_PYTHON/ACT7'

# comprobar si la carpeta ya existe
if not os.path.exists(ruta_folder):
    os.makedirs(ruta_folder)  # crea la carpeta si no existe
    print(f"Se creo la carpeta : {ruta_folder.split('/')[4]}")
else:
    print(f"La carpeta {ruta_folder.split('/')[4]} ya existe y esta lista para su uso.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
La carpeta L_PYTHON ya existe y esta lista para su uso.


## Importacion de modulo con funciones genericas utiles

In [65]:
import sys
sys.path.append(ruta_folder + '/modulos')
import utils

## Funciones utilitarias para la actividad 7

In [66]:
from google.colab import files

def obtener_ruta_archivo():
    """
    Esta funcion pide un archivo (existente o subir uno nuevo)
    y retorna la ruta
    """

    opcion = utils.get_num_in_range("1. Archivo existente(drive)\n2. Subir archivo\n0. Volver atras\n> Ingresa opcion: ", 0, 2)
    ruta_archivo = None

    # construccion ruta completa archivo
    if opcion == 1:
        print("\n+--- Archivos disponibles: ---+")
        for file_name in os.listdir(ruta_folder):
            print(file_name)

        filename = input("> Ingresa nombre del archivo a cargar (con extensión): ")
        ruta_archivo = os.path.join(ruta_folder, filename)

        if not os.path.exists(ruta_archivo):
            raise FileNotFoundError(f"El archivo '{filename}' no existe en la ruta especificada.")

    elif opcion == 2:
        uploaded_files = files.upload()
        if not uploaded_files:
            raise ValueError("No se subió ningún archivo.")
        ruta_archivo = list(uploaded_files.keys())[0]

    return ruta_archivo


def leer_archivo(file):
    """
    Lee el archivo según su extensión y devuelve el contenido.
    """
    _, extension = os.path.splitext(file)

    lectores = {
        '.xlsx': lambda f: pd.read_excel(f, sheet_name=0),
        '.csv': pd.read_csv,
        '.txt': lambda f: open(f, 'r').read(),
        '.md': lambda f: open(f, 'r').read(),
    }

    lector = lectores.get(extension)
    if not lector:
        raise TypeError(f"Tipo de archivo '{extension}' no admitido.")

    try:
        return lector(file)
    except Exception as e:
        print(f"Error al leer el archivo: {e}")

def str_to_list_of_dicts(contenido_txt):
    """
    Recibe el contenido de un archivo txt como una cadena de texto,
    lo carga en una lista de diccionarios y la devuelve.
    """
    datos = []
    for linea in contenido_txt.splitlines():
        linea = linea.strip()
        if linea:
            datos_limpios = linea.split() # lista con cada palabra de la linea actual
            datos_limpios.pop(0)  # elimina el consecutivo

            if len(datos_limpios) != 7:
                raise TypeError("Formato de archivo incorrecto.")

            # conversion a entero de los valores numericos
            for i, dato in enumerate(datos_limpios):
                try:
                    datos_limpios[i] = int(dato)
                except ValueError:
                    pass
            datos.append(list_to_dict(datos_limpios))  # convierte a diccionario y agrega a la lista
    return datos


def list_to_dict(lista):
    """
    convierte una lista, en un diccionario
    """

    keys = ['id', 'nombre', 'apPat', 'apMat', 'sexo', 'edad', 'puesto']
    diccionario = {}
    for i, dato in enumerate(lista):
        diccionario[keys[i]] = dato
    return diccionario


def tabular_datos_detalle(empleados):
    """
    Esta funcion itera la lista y formatea los datos en forma de
     tabla con un diseño mas atractivo, y lo regresa en una variable str
    """

    encabezado = '|   No.  |   ID   |   NOMBRE     |    APPAT     |     APMAT     |    SEXO    | EDAD |    PUESTO     |\n'
    linea = '+' + '-' * 99 + '+\n'

    datos = [linea, encabezado, linea]

    for i, empl in enumerate(empleados, start=1):
        fila = (
            f"| {i:^6d} "
            f"| {empl['id']:^7d}"
            f"| {empl['nombre']:^11s}  "
            f"| {empl['apPat']:^11s}  "
            f"| {empl['apMat']:^11s}   "
            f"| {empl['sexo']:^9s}  "
            f"| {empl['edad']:^3d}  "
            f"| {empl['puesto']:^13s} |\n"
        )
        datos.append(fila)
    return ''.join(datos) + linea

def tabular_datos(empleados):
    """
    Esta funcion itera la lista y formatea los datos
    en forma de tabla cargable, y lo regresa en una variable str
    """
    datos = []

    for i, empl in enumerate(empleados, start=1):
        fila = (
            f"{i:^6d} "
            f"{empl['id']:^7d}"
            f"{empl['nombre']:^11s}  "
            f"{empl['apPat']:^11s}  "
            f"{empl['apMat']:^11s}   "
            f"{empl['sexo']:^9s}  "
            f"{empl['edad']:^3d}  "
            f"{empl['puesto']:^13s}\n"
        )
        datos.append(fila)
    return ''.join(datos)

## Menu's

In [67]:
def menu_principal():
    print("+--- MENU ACCIONES ---+")
    print("1. Agregar (AUTO 10)")
    print("2. Eliminar {ID}")
    print("3. Imprimir lista (tabla)")
    print("4. Buscar {ID}")
    print("5. Buscar {ApPat}")
    print("6. Ordenar")
    print("7. Generar archivo")
    print("8. Cargar archivo")
    print("9. Imprimir archivo")
    print("10. Borrar la lista")
    print("0. Salir")
    return utils.get_num_in_range("> : ", 0, 10)

def menu_archivo(msg):
    print('1. Excel (.xlsx)')
    print('2. Texto (.txt)')
    print('3. CSV (.csv)')
    print('4. Markdown (.md)')
    print("0. Volver")
    return utils.get_num_in_range(msg, 0, 4)

## 1. Agregar (automatico 10)


In [68]:
def agregar_automatico(empleados, n = 10):
    for i in range(n):
        id = utils.generar_id_unico(empleados, lambda: utils.gen_num_aleatorio(300000, 399999))
        sexo = utils.random.choice(['HOMBRE', 'MUJER'])
        nombre = utils.generar_nombre(sexo)
        apPat, apMat = utils.generar_apellidos()
        edad = utils.gen_num_aleatorio(16, 65);
        puesto = utils.generar_puesto()
        empleados.append({'id': id, 'nombre':nombre, 'apPat':apPat, 'apMat':apMat,
                        'sexo':sexo, 'edad':edad, 'puesto':puesto})
    else:
        print(f"Se han agregado {n} registros a la lista.")

## 2. Eliminar {ID}

In [69]:
def eliminar(empleados):
    id = utils.get_num_in_range("Ingresa ID a eliminar: ", 300000, 399999)
    indice = utils.busq_secuencial(empleados, id)

    if indice != -1:
        print(f"Empleado encontrado en la posicion : {indice}")
        utils.imprimir_registro(empleados[indice])
        if utils.get_num_in_range("¿Desea eliminar? (1.Si - 2.No): ", 1, 2) == 1:
            empleados.pop(indice)
            print(f"Empleado eliminado.")
    else:
        print(f"No se encontro el empleado con ID {id}")

## 3. Imprimir lista (tabla)

In [70]:
def imprimir_todo(empleados):
    if empleados:
        print(tabular_datos_detalle(empleados))
    else:
        print("La lista esta vacia.")

## 4. Buscar {ID}

In [71]:
def buscar(empleados):
    id = utils.get_num_in_range("Ingresa ID a buscar: ", 300000, 399999)
    indice = utils.busq_secuencial(empleados, id)
    if indice != -1:
        print(f"Empleado encontrado en la posicion : {indice}")
        utils.imprimir_registro(empleados[indice])
    else:
        print(f"No se encontro el empleado con ID {id}")

## 5. Buscar {ApPat}

In [72]:
def buscar_por_apellido(empleados):
    def obtener_coincidencias(lista, valor, key='apPat'):
        encontrados = []
        for i, item in enumerate(lista):
            if item[key] == valor:
                encontrados.append(empleados[i])
        return encontrados

    appat = input("Ingresa apellido a buscar: ").upper()
    coincidencias = obtener_coincidencias(empleados, appat)

    if coincidencias:
        print(f"Se encontraron los siguientes trabajadores con el apellido {appat}: ")
        print(tabular_datos_detalle(coincidencias))
    else:
        print(f"No se encontraron trabajadores con el apellido paterno : {appat}")

## 6. Ordenar {ID}

In [73]:
def ordenar(empleados):
    if not utils.esta_ordenada(empleados):
        empleados.sort(key=lambda x: x['id'])
        print("Se ha ordenado la lista.")
    else:
        print("No se ordeno, ya estaba ordenada.")

## 7. Generar archivo (excel, txt, csv, markdown)

In [74]:
import pandas as pd

def generar_archivo(empleados):
    opcion = menu_archivo("> Ingresa tipo archivo [1-4]: ")
    if opcion == 0: return

    file_ext = {
        1: '.xlsx',
        2: '.txt',
        3: '.csv',
        4: '.md'
    }

    filename = input("> Ingresa nombre para el archivo: ") + file_ext[opcion]
    ruta_destino = os.path.join(ruta_folder, filename)

    datos = pd.DataFrame(empleados)

    if opcion == 1:
        datos.to_excel(ruta_destino, index=False)
    elif opcion == 2:
        with open(os.path.join(ruta_destino), "w") as file:
            file.write(tabular_datos(empleados))
    elif opcion == 3:
        datos.to_csv(ruta_destino, index=False)
    elif opcion == 4:
        datos.to_markdown(ruta_destino, index=False)
    print(f"Archivo {filename} generado correctamente.")

## 8. Cargar archivo

In [75]:
def cargar_archivo_lista(empleados):
    try:
        ruta_archivo = obtener_ruta_archivo()

        _, extension = os.path.splitext(ruta_archivo)

        if extension not in ['.xlsx', '.csv', '.txt']:
            print("Formato de archivo no compatible. Usa (.xlsx, .csv, .txt).")
            return

        contenido_archivo = leer_archivo(ruta_archivo)

        if contenido_archivo is not None:
            empleados.clear()

            # convierte el contenido del archivo al formato de lista de diccionarios
            if extension in ['.xlsx', '.csv']:
                datos_a_cargar = contenido_archivo.to_dict(orient='records')
            elif extension == '.txt':
                datos_a_cargar = str_to_list_of_dicts(contenido_archivo)

            empleados.extend(datos_a_cargar)
            print("Se ha cargado el contenido del archivo en la lista.")
        else:
            print("Error: No se pudo leer el archivo.")

    except (FileNotFoundError, ValueError, TypeError) as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")

## 9. Imprimir archivo

In [76]:
def imprimir_archivo():
    try:
        ruta_archivo = obtener_ruta_archivo()
        print(leer_archivo(ruta_archivo))
    except (FileNotFoundError, ValueError, TypeError) as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")

## 10. Borrar toda la lista

In [77]:
def limpiar_lista(lista):
    if utils.get_num_in_range("¿Seguro que desea eliminar la lista?: (1.Si - 2.No) : ", 1, 2) == 1:
        lista.clear()
        print("Se ha borrado la lista.")

## Main

In [79]:
if __name__ == '__main__':
    empleados = []

    acciones = {
        1: agregar_automatico,
        2: eliminar,
        3: imprimir_todo,
        4: buscar,
        5: buscar_por_apellido,
        6: ordenar,
        7: generar_archivo,
        8: cargar_archivo_lista,
        9: imprimir_archivo,
        10: limpiar_lista
    }

    while True:
        opcion = menu_principal()
        print()
        if opcion in acciones:
            funcion = acciones[opcion]
            funcion(empleados) if funcion != imprimir_archivo else funcion()
        else:
            print("Has salido.")
            break
        print()

+--- MENU ACCIONES ---+
1. Agregar (AUTO 10)
2. Eliminar {ID}
3. Imprimir lista (tabla)
4. Buscar {ID}
5. Buscar {ApPat}
6. Ordenar
7. Generar archivo
8. Cargar archivo
9. Imprimir archivo
10. Borrar la lista
0. Salir
> : 8

1. Archivo existente(drive)
2. Subir archivo
0. Volver atras
> Ingresa opcion: 1

+--- Archivos disponibles: ---+
prueba_texto.txt
prueba_markdown.md
modulos
registros.xlsx
cargatxt.txt
pruebatex.txt
prueba2.txt
pruebacsv.csv
prueba-pathjoin.txt
sadas.xlsx
> Ingresa nombre del archivo a cargar (con extensión): pruebacsv.csv
Se ha cargado el contenido del archivo en la lista.

+--- MENU ACCIONES ---+
1. Agregar (AUTO 10)
2. Eliminar {ID}
3. Imprimir lista (tabla)
4. Buscar {ID}
5. Buscar {ApPat}
6. Ordenar
7. Generar archivo
8. Cargar archivo
9. Imprimir archivo
10. Borrar la lista
0. Salir
> : 3

+---------------------------------------------------------------------------------------------------+
|   No.  |   ID   |   NOMBRE     |    APPAT     |     APMAT     |    